# TextGAN

In [ ]:
#Solo si se está usando Google Colab
#from google.colab import drive
#drive.mount('/content/gdrive/')

In [ ]:
#Solo si se está usando Google Colab
#Solo ejecutar una vez, es para clonar el repo en drive (para tener las salidas generadas guardadas en el drive en caso de que se apague la maquina).
!git clone -b Thesis https://github.com/YuiZh0u/TextGAN-PyTorch.git

## Instalación de requisitos

In [ ]:
!cd TextGAN-PyTorch && pip install -r requirements.txt

In [ ]:
!apt-get install libboost-all-dev

In [ ]:
!apt-get install build-essential cmake

In [ ]:
!git clone https://github.com/kpu/kenlm.git

In [ ]:
!cd kenlm && mkdir -p build && cd build && cmake .. && make -j 4

In [5]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/jfrez/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [7]:
import torch
torch.cuda.empty_cache()

In [8]:
# !cp /content/gdrive/MyDrive/word2vec/covid_tweets_equal16.txt /content/gdrive/MyDrive/TextGAN-PyTorch/dataset/testdata/

In [9]:
from io import StringIO
import sys

from IPython.core import magic_arguments
from IPython.core.magic import Magics, cell_magic, magics_class
from IPython.utils.capture import CapturedIO


class Tee(StringIO):
    def __init__(self, initial_value='', newline='\n', stream=None):
        self.stream = stream
        super().__init__(initial_value, newline)
    
    def write(self, data):
        if self.stream is not None:
            self.stream.write(data)
        
        super().write(data)


class capture_and_print_output(object):
    stdout = True
    stderr = True
    display = True
    
    def __init__(self, stdout=True, stderr=True, display=True):
        self.stdout = stdout
        self.stderr = stderr
        self.display = display
        self.shell = None
    
    def __enter__(self):
        from IPython.core.getipython import get_ipython
        from IPython.core.displaypub import CapturingDisplayPublisher
        from IPython.core.displayhook import CapturingDisplayHook
        
        self.sys_stdout = sys.stdout
        self.sys_stderr = sys.stderr
        
        if self.display:
            self.shell = get_ipython()
            if self.shell is None:
                self.save_display_pub = None
                self.display = False
        
        stdout = stderr = outputs = None
        if self.stdout:
            stdout = sys.stdout = Tee(stream=sys.stdout)
        if self.stderr:
            stderr = sys.stderr = Tee(stream=sys.stderr)
        if self.display:
            self.save_display_pub = self.shell.display_pub
            self.shell.display_pub = CapturingDisplayPublisher()
            outputs = self.shell.display_pub.outputs
            self.save_display_hook = sys.displayhook
            sys.displayhook = CapturingDisplayHook(shell=self.shell,
                                                   outputs=outputs)
        
        return CapturedIO(stdout, stderr, outputs)
    
    def __exit__(self, exc_type, exc_value, traceback):
        sys.stdout = self.sys_stdout
        sys.stderr = self.sys_stderr
        if self.display and self.shell:
            self.shell.display_pub = self.save_display_pub
            sys.displayhook = self.save_display_hook


@magics_class
class CustomMagics(Magics):
    @magic_arguments.magic_arguments()
    @magic_arguments.argument('output', type=str, default='', nargs='?')
    @magic_arguments.argument('--no-stderr', action='store_true')
    @magic_arguments.argument('--no-stdout', action='store_true')
    @magic_arguments.argument('--no-display', action='store_true')
    @cell_magic
    def tee(self, line, cell):
        args = magic_arguments.parse_argstring(self.tee, line)
        out = not args.no_stdout
        err = not args.no_stderr
        disp = not args.no_display
        with capture_and_print_output(out, err, disp) as io:
            self.shell.run_cell(cell)
        if args.output:
            self.shell.user_ns[args.output] = io

In [10]:
from IPython import get_ipython

get_ipython().register_magics(CustomMagics)

In [ ]:
%%tee output
!cd TextGAN-PyTorch/run && python3 run_seqgan.py 3 0

job_id: 3, gpu_id: 0
> training arguments:
>>> if_test: 0
>>> run_model: seqgan
>>> k_label: 2
>>> dataset: covid_tweets
>>> model_type: vanilla
>>> loss_type: rsgan
>>> mu_type: ragan
>>> eval_type: Ra
>>> d_type: Ra
>>> if_real_data: 1
>>> cuda: 1
>>> device: 0
>>> devices: 0
>>> shuffle: 0
>>> gen_init: normal
>>> dis_init: uniform
>>> n_parent: 1
>>> eval_b_num: 8
>>> lambda_fq: 1.0
>>> lambda_fd: 0.0
>>> d_out_mean: True
>>> freeze_dis: False
>>> freeze_clas: False
>>> use_all_real_fake: False
>>> use_population: False
>>> samples_num: 10000
>>> vocab_size: 29329
>>> mle_epoch: 5
>>> clas_pre_epoch: 10
>>> adv_epoch: 3
>>> inter_epoch: 15
>>> batch_size: 64
>>> max_seq_len: 66
>>> start_letter: 1
>>> padding_idx: 0
>>> gen_lr: 0.01
>>> gen_adv_lr: 0.0001
>>> dis_lr: 0.0001
>>> clip_norm: 5.0
>>> pre_log_step: 10
>>> adv_log_step: 1
>>> train_data: dataset/covid_tweets.txt
>>> test_data: dataset/testdata/covid_tweets_test.txt
>>> temp_adpt: exp
>>> evo_temp_step: 1
>>> temperature:

In [ ]:
# %%tee output
# !cd TextGAN-PyTorch/run && python3 run_catgan.py 6 0

In [ ]:
from datetime import datetime
filename = datetime.now().strftime("%Y%m%d_%H%M")
with open('outputs_pynb/output{}.txt'.format(filename), 'w') as f:
    f.write(output.stdout)

In [ ]:
# with open('output08.txt', 'w') as f:
#     f.write(output.stdout)

In [ ]:
!nvidia-smi

In [ ]:
!echo "hola"

In [ ]:
!cd TextGAN-PyTorch/utils && python3 visualization-Copy1.py